#  Problème 2: Mesure du Risque

**Collecte de données CRSP (2012-2023)**


In [ ]:

# FONCTIONS POUR TÉLÉCHARGER LES DONNÉES CRSP


import wrds
import pandas as pd
import time

# Connexion WRDS réutilisable
_db_connection = None

def get_wrds_connection(username=None):
    """Obtient une connexion WRDS réutilisable"""
    global _db_connection
    
    if _db_connection is None:
        if username:
            _db_connection = wrds.Connection(wrds_username=username)
        else:
            _db_connection = wrds.Connection()
    
    return _db_connection

def fetch_crsp(permnos, start='2012-01-03', end='2023-01-03', out_path='crsp_data.csv'):
    """
    Télécharge les données CRSP pour une liste de PERMNOs
    
    PARAMÈTRES:
    -----------
    permnos : list - Liste des PERMNOs à télécharger
    start : str - Date de début (défaut: '2012-01-03')
    end : str - Date de fin (défaut: '2023-01-03')
    out_path : str - Fichier de sauvegarde (défaut: 'crsp_data.csv')
    
    RETOURNE:
    ---------
    pandas.DataFrame - Données CRSP avec colonnes: date, permno, prc, ret, vol
    """
    
    try:
        db = get_wrds_connection()
        
        sql = f"""
        SELECT date, permno, prc, ret, vol
        FROM crsp.dsf
        WHERE permno IN ({','.join(map(str, permnos))})
          AND date BETWEEN '{start}' AND '{end}'
        ORDER BY permno, date
        """
        
        df = db.raw_sql(sql, date_cols=['date'])
        df.to_csv(out_path, index=False)
        
        return df
        
    except Exception as e:
        print(f"Erreur: {e}")
        raise

def close_connection():
    """Ferme la connexion WRDS"""
    global _db_connection
    if _db_connection:
        _db_connection.close()
        _db_connection = None

In [60]:

# Liste complète des 10 titres requis
permnos_confirmes = [
    # INDICE DE MARCHÉ
    84398,              # SPY - SPDR S&P 500 ETF Trust
    
    # SECTEUR FINANCIER
    52476,              # EFX - Equifax Inc. 
    47896,              # JPM - JPMorgan Chase & Co.
    90162,              # GNW - Genworth Financial Inc.
    
    # SECTEUR TECHNOLOGIQUE
    75828,              # EA - Electronic Arts Inc.
    79094,              # JBL - Jabil Inc.
    90319,              # GOOGL - Alphabet Inc. (Class A)
    
    # SECTEUR ÉNERGÉTIQUE
    75825,              # EOG - EOG Resources Inc.
    93380,              # JKS - JinkoSolar Holding Co.
    92619,              # GTE - Gran Tierra Energy Inc.
]

In [63]:
import os

# Verifier si les donnees existent deja
if os.path.exists('donnees_probleme2.csv'):
    print("Chargement des donnees existantes...")
    df = pd.read_csv('donnees_probleme2.csv')
    df['date'] = pd.to_datetime(df['date'])
else:
    print("Telechargement des donnees CRSP...")
    df = fetch_crsp(permnos_confirmes, out_path='donnees_probleme2.csv')
    
    # Mapping PERMNO -> Ticker et ajout de la colonne
    ticker_map = {
        84398: 'SPY', 52476: 'EFX', 47896: 'JPM', 90162: 'GNW',
        75828: 'EA', 79094: 'JBL', 90319: 'GOOGL',
        75825: 'EOG', 93380: 'JKS', 92619: 'GTE'
    }
    
    df['ticker'] = df['permno'].map(ticker_map)
    df = df[['date', 'permno', 'ticker', 'prc', 'ret', 'vol']]
    df.to_csv('donnees_probleme2.csv', index=False)

# Creation des fichiers individuels si necessaire
os.makedirs('donnees_individuelles', exist_ok=True)

ticker_map = {
    84398: 'SPY', 52476: 'EFX', 47896: 'JPM', 90162: 'GNW',
    75828: 'EA', 79094: 'JBL', 90319: 'GOOGL',
    75825: 'EOG', 93380: 'JKS', 92619: 'GTE'
}

for permno in permnos_confirmes:
    ticker = ticker_map[permno]
    fichier = f'donnees_individuelles/{ticker}_{permno}.csv'
    if not os.path.exists(fichier):
        data_titre = df[df['permno'] == permno]
        data_titre.to_csv(fichier, index=False)

print(f"Donnees: {len(df):,} observations pour {df['permno'].nunique()} titres")
print("\nObservations par titre:")
obs_par_titre = df.groupby(['permno', 'ticker']).size().reset_index(name='observations')
for _, row in obs_par_titre.iterrows():
    print(f"  {row['ticker']} (PERMNO {row['permno']}): {row['observations']:,} observations")

Chargement des donnees existantes...
Donnees: 27,690 observations pour 10 titres

Observations par titre:
  JPM (PERMNO 47896): 2,769 observations
  EFX (PERMNO 52476): 2,769 observations
  EOG (PERMNO 75825): 2,769 observations
  EA (PERMNO 75828): 2,769 observations
  JBL (PERMNO 79094): 2,769 observations
  SPY (PERMNO 84398): 2,769 observations
  GNW (PERMNO 90162): 2,769 observations
  GOOGL (PERMNO 90319): 2,769 observations
  GTE (PERMNO 92619): 2,769 observations
  JKS (PERMNO 93380): 2,769 observations
